In [ ]:
import sklearn
import nltk
import nltk.classify.util
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk import sent_tokenize
import string
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix, f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
print(__doc__)
import warnings
warnings.filterwarnings('ignore')
#from nltk.classify import DecisionTree

In [ ]:
TRAIN_TEXT_DATA_DIR ='../input/clicknonclickbait/clicknonclickbait.csv'

In [ ]:
############################################
# Get labels
############################################
print('Loading Training ClickBait and Non-ClickBait data.')
out = pd.read_csv(TRAIN_TEXT_DATA_DIR, encoding='utf-8')
prediction_labels = ["ClickBait","NonClickBait"]
print('Getting Comment Labels.')
labels = out[prediction_labels].values
out.head()

In [ ]:
comments = out['Comments'].values
print(comments[10])

In [ ]:
# Creating histogram
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(out.index, bins = [100, 1000, 10000, 20000, 30000])
# Show plot
plt.show()

In [ ]:
stop_words = set(stopwords.words('english'))
wrds= [nltk.word_tokenize(word) for word in comments]
stop=[]
for w in  wrds:
    for i in w:
        stop.append(i)
filtered_sentence = [w for w in stop if w.lower() in stop_words]
txt = " ".join(rev for rev in set(filtered_sentence))
wordcloud = WordCloud(min_font_size = 10,background_color="white",max_words=1000).generate(txt)
plt.figure(figsize = (8, 8), facecolor=None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
before = len(set(filtered_sentence))
print(before)

In [ ]:
#removing the stopwords 
def remove_stopwords():
        processed_word_list = out['Comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        return processed_word_list

In [ ]:
filtered = remove_stopwords()
filtered[350]

In [ ]:
wrds2= [nltk.word_tokenize(word) for word in filtered]
stops=[]
for w2 in  wrds2:
    for j in w2:
        stops.append(j)
filtered_sent = [w for w in stops if w.lower() in stop_words]
txts = " ".join(re for re in set(filtered_sent))
wordcld = WordCloud(min_font_size = 10,background_color="white",max_words=1000).generate(txts)
plt.figure(figsize = (8, 8), facecolor=None)
plt.imshow(wordcld, interpolation='bilinear')
plt.axis("off")
plt.show()
after = len(set(filtered_sent))

In [ ]:
activities = ['Before \nremoval','After\nremoval']
# portion covered by each label
slices = [before, after] 
# color for each label
colors = ['g', 'c']
# plotting the pie chart
plt.pie(slices, labels = activities , colors=colors, 
        startangle=90, shadow = True, explode = ( 0.1, 0),
        radius = 1.2, autopct = '%1.1f%%')
plt.title('Overview of Stopwords Data')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
data = [before, after]
plt.bar(['Before \nremoval of stopwords','After\nremoval of stopwords'],data, color ='Blue')
plt.ylabel("Number of Stopwords")
plt.title("Overview of Stopwords Data")
plt.show()

In [ ]:
#To remove punctuations and tokenziation to to remove whitespaces
text = [sent_tokenize(word) for word in filtered]
sentences = text
print("Before:-> ",sentences[2])
string_text= [str(word).translate(str.maketrans("","",string.punctuation)) for word in sentences]
print("After:-> ",string_text[2])

In [ ]:
#Lemmitizing the sentences
lemmatizer = WordNetLemmatizer()
lem=[]
lem_word=[]
for word in string_text:
    word_list = nltk.word_tokenize(word) 
    lem_word.append([lemmatizer.lemmatize(w) for w in word_list])
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    lem.append(lemmatized_output)
print(lem[350])

In [ ]:
X= pd.DataFrame({'Comments':lem})
Y= pd.DataFrame(labels)
Y.columns = ['ClickBait','NonClickBait']
final_datas =pd.concat([X, Y], axis=1, join='inner')
final_datas.to_csv('final.csv', encoding='utf-8',index=False)
final_data = shuffle(final_datas)
clk=final_data.loc[final_data["ClickBait"] == True]
nonc=final_data.loc[final_data["NonClickBait"] == True]


In [ ]:
wrds3= [nltk.word_tokenize(word) for word in clk["Comments"]]
stopped=[]
for w2 in  wrds3:
    for j in w2:
        stopped.append(j)
filtered_sent = [w for w in stopped if w.lower() in stop_words]
txts = " ".join(re for re in set(filtered_sent))
wordcld = WordCloud(min_font_size = 10,background_color="white",max_words=1000).generate(txts)
plt.figure(figsize = (8, 8), facecolor=None)
plt.imshow(wordcld, interpolation='bilinear')
plt.axis("off")
plt.show()
clkstop = len(set(filtered_sent))

In [ ]:
wrds3= [nltk.word_tokenize(word) for word in nonc["Comments"]]
stopped=[]
for w2 in  wrds3:
    for j in w2:
        stopped.append(j)
filtered_sen = [w for w in stopped if w.lower() in stop_words]
txted = " ".join(re for re in set(filtered_sen))
wordcld = WordCloud(min_font_size = 10,background_color="white",max_words=1000).generate(txted)
plt.figure(figsize = (8, 8), facecolor=None)
plt.imshow(wordcld, interpolation='bilinear')
plt.axis("off")
plt.show()
noncstop = len(set(filtered_sen))

In [ ]:
fig = plt.figure()
data = [clkstop, noncstop]
plt.bar(['ClickBait','NonClickBait'],data, color ='Yellow')
plt.ylabel("Number of Stopwords")
plt.title("Overview of Stopwords Data")
plt.show()

In [ ]:
activities = ["ClickBait","NonClickBait"] 
# portion covered by each label
slices = [clk.shape[0], nonc.shape[0]] 
# color for each label
colors = ['g', 'c']
# plotting the pie chart
plt.pie(slices, labels = activities, colors=colors, 
        startangle=90, shadow = True, explode = ( 0.1, 0),
        radius = 1.2, autopct = '%1.1f%%')
plt.title('Overview of Data')
plt.legend()
plt.show()

In [ ]:
trains =final_data['Comments'].values
tests = final_data[prediction_labels].values
X_train, X_test, y_train, y_test = train_test_split(trains,tests, random_state=42, test_size=0.33, shuffle=True)

In [ ]:
activities = ["Training Data","Testing Data"] 
# portion covered by each label
slices = [X_train.shape[0], X_test.shape[0]] 
# color for each label
colors = ['y', 'b']
# plotting the pie chart
plt.pie(slices, labels = activities, colors=colors, 
        startangle=90, shadow = True, explode = ( 0.1, 0),
        radius = 1.2, autopct = '%1.1f%%')
plt.title('Training & Testing Data')
plt.legend()
plt.show()

In [ ]:
activities = ["Training Data","Testing Data"] 
# portion covered by each label
slices = [y_train.shape[0], y_test.shape[0]] 
# color for each label
colors = ['g', 'c']
# plotting the pie chart
plt.pie(slices, labels = activities, colors=colors, 
        startangle=90, shadow = True, explode = ( 0.1, 0),
        radius = 1.2, autopct = '%1.1f%%')
plt.title('Validation Data')
plt.legend()
plt.show()

In [ ]:
names = ["Logistic Regression", "Linear SVM", "RBF SVM",
         "Decision Tree","Voting Classifier"]
classifiers = [
    OneVsRestClassifier(LogisticRegression(solver='sag'),n_jobs=1),
    OneVsRestClassifier(SVC(kernel="linear", C=0.025,probability=True)),
    OneVsRestClassifier(SVC(gamma=2, C=1,probability=True)),
    OneVsRestClassifier(DecisionTreeClassifier(max_depth=5)),
    OneVsRestClassifier(VotingClassifier(estimators=[('Logistic ', OneVsRestClassifier(LogisticRegression(solver='sag'),n_jobs=1)), ('SVM Linear', OneVsRestClassifier(SVC(kernel="linear", C=0.025,probability=True))), ('SVM RBF', OneVsRestClassifier(SVC(gamma=2, C=1,probability=True))),('Decision Tree',OneVsRestClassifier(DecisionTreeClassifier(max_depth=5)))],
                        voting='soft', weights=[2, 1, 2, 5]))]

In [ ]:
# iterate over classifiers
dicts = {}
for name, clf in zip(names, classifiers):
    print("Fitting on : ",name)
    LogReg_pipeline = Pipeline([('tfidf', TfidfVectorizer()),('clf', clf),])
    LogReg_pipeline.fit(X_train, y_train)
    dicts[name]= LogReg_pipeline
    print('Loaded',name)    

In [ ]:
for i,j in dicts.items():
    prediction = j.predict(X_test)
    print('Test accuracy for {} is {}'.format(i, accuracy_score(y_test, prediction)))

In [ ]:
probas1=[]
probas2=[]
probas3=[]
for i,j in dicts.items():
    prediction = j.predict(X_test)
    print("------Metrics Evaluation for {} --------".format(i))
    print('\nF1 Score is {}'.format(f1_score(y_test, prediction, average='weighted')))
    print('Recall Score is {}'.format(recall_score(y_test, prediction, average='micro')))
    print('Precision Score is {}'.format(precision_score(y_test, prediction, average='micro')))
    print("")
    probas1.append(round(f1_score(y_test, prediction, average='weighted'),2))
    probas2.append(round(recall_score(y_test, prediction, average='micro'),2))
    probas3.append(round(precision_score(y_test, prediction, average='micro'),2))

In [ ]:
# get class probabilities for the first sample in the dataset(E.G ClickBait & NonClickBait)
# plotting
N = 5  # number of groups
ind = np.arange(N)  # group positions
width = 0.25  # bar width
fig, ax = plt.subplots()
# bars for classifier 1-5
p1 = ax.bar(ind, np.hstack(probas1), width, color='green', edgecolor='k')
p2 = ax.bar(ind + width, np.hstack(probas2), width, color='lightgreen', edgecolor='k')
p3 = ax.bar(ind + width + width, np.hstack(probas3), width, color='yellow', edgecolor='k')
# bars for VotingClassifier
p4 = ax.bar(ind, [0, 0, 0, 0,  probas1[-1]], width, color='blue', edgecolor='k')
p5 = ax.bar(ind + width, [0, 0, 0, 0, probas2[-1]], width, color='steelblue', edgecolor='k')
p6 = ax.bar(ind + width + width, [0, 0, 0, 0, probas3[-1]], width, color='red', edgecolor='k')
# plot annotations
plt.axvline(3.8, color='k', linestyle='dashed')
ax.set_xticks(ind + width)
ax.set_xticklabels(['LogisticRegression',
                    'Linear SVM',
                    'RBF SVM',
                    'Decision Tree',
                    'VotingClassifier\n(average probabilities)'],
                   rotation=40,
                   ha='right')
plt.ylim([0, 1])
plt.title('Metrics Evaluation for different classifiers')
plt.legend([p1[0], p2[0],p3[0]], ['F1', 'Recall','Precision'], loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
probas = [j.predict(X_test) for i,j in dicts.items()]
# get class probabilities for the sample in the dataset(E.G ClickBait & NonClickBait)
class1_1 = [accuracy_score(y_test, prediction)*100 for prediction in probas]
# plotting
N = 5  # number of groups
ind = np.arange(N)  # group positions
width = 0.35  # bar width
fig, ax = plt.subplots()
# bars for classifier 1-5
p1 = ax.bar(ind, class1_1[0:5], width, color='purple', edgecolor='k')
# bars for VotingClassifier
p2 = ax.bar(ind, [0, 0, 0, 0,  class1_1[-1]], width, color='blue', edgecolor='k')
# plot annotations
plt.axvline(3.8, color='k', linestyle='dashed')
ax.set_xticks(ind + width)
ax.set_xticklabels(['LogisticRegression',
                    'Linear SVM',
                    'RBF SVM',
                    'Decision Tree',
                    'VotingClassifier'],
                   rotation=40,
                   ha='right')
plt.ylim([0, 100])
plt.title('Test accuracy for classifiers')
plt.tight_layout()
plt.show()

In [ ]:
test = ["I totally block the account and click never show again when I see that"]
probas= [(i,j.predict_proba(test)) for i,j in dicts.items()]
dat = []
for pr in probas:
    data={'Classifier':pr[0],'Comment':test,'ClickBait':round(pr[1][0,0],2),'NonClickBait':round(pr[1][0,1],2)}
    dat.append(data)
results=pd.DataFrame(dat)
results.head()

In [ ]:
probas = [j.predict_proba(test) for i,j in dicts.items()]
# get class probabilities for the first sample in the dataset(E.G ClickBait & NonClickBait)
class1_1 = [pr[0, 0]*100 for pr in probas]
class2_1 = [pr[0, 1]*100 for pr in probas]

# plotting
N = 5  # number of groups
ind = np.arange(N)  # group positions
width = 0.35  # bar width
fig, ax = plt.subplots()
# bars for classifier 1-5
p1 = ax.bar(ind, np.hstack(([class1_1[:-1], [0]])), width, color='green', edgecolor='k')
p2 = ax.bar(ind + width, np.hstack(([class2_1[:-1], [0]])), width, color='lightgreen', edgecolor='k')
# bars for VotingClassifier
p3 = ax.bar(ind, [0, 0, 0, 0,  class1_1[-1]], width, color='blue', edgecolor='k')
p4 = ax.bar(ind + width, [0, 0, 0, 0, class2_1[-1]], width, color='steelblue', edgecolor='k')
# plot annotations
plt.axvline(3.8, color='k', linestyle='dashed')
ax.set_xticks(ind + width)
ax.set_xticklabels(['LogisticRegression',
                    'Linear SVM',
                    'RBF SVM',
                    'Decision Tree',
                    'VotingClassifier\n(average probabilities)'],
                   rotation=40,
                   ha='right')
plt.ylim([0, 100])
plt.title('Class probabilities for 1 Sample by different classifiers')
plt.legend([p1[0], p2[0]], ['ClickBait', 'NonClickBait'], loc='upper left')
plt.tight_layout()
plt.show()